In [41]:
import pandas as pd
import numpy as np
import re

In [42]:
data = pd.read_excel(io="../data/input/new_taxonomy.xlsx", sheetname="Sheet1").fillna('')
data.head(10)

,Topic,Subtopic,Measuretitle
0,Housing and Living Standards,Housing,Home ownership in England
1,Housing and Living Standards,Housing,People who get financial help from their paren...
2,Housing and Living Standards,Housing,Average age of people who buy a home
3,Housing and Living Standards,Housing,Home buyer deposits in England
4,Housing and Living Standards,Housing,People living in social housing in England
5,Housing and Living Standards,Housing,Reasons for moving into social housing
6,Housing and Living Standards,Housing,How long people live in social housing
7,Housing and Living Standards,Housing,Different routes into social housing
8,Housing and Living Standards,Housing,Vulnerable people living in social housing
9,Housing and Living Standards,Housing,Access to social housing


In [43]:
output_columns = ['name', 'parent_name', 'uri', 'parent_uri', 'description', 'level', 'source', ]
output = pd.DataFrame(columns=output_columns)

In [44]:
t0_output = pd.DataFrame(columns=output_columns)
t0_output.loc[len(t0_output)]=['Homepage','','/','', 'Racial Disparity Audit Homepage', 'T0', '']
t0_output

,name,parent_name,uri,parent_uri,description,level,source
0,Homepage,,/,,Racial Disparity Audit Homepage,T0,


In [45]:
def to_uri(astr):
    return re.sub(r'\W+', '', astr).lower()

In [46]:
t1_output = pd.DataFrame(columns=output_columns)

unique_topics = data.Topic.unique()
for topic in unique_topics:
    t1_output.loc[len(t1_output)]=[topic,'Homepage','/' + to_uri(topic),'/', topic, 'T1', '']

t1_output

,name,parent_name,uri,parent_uri,description,level,source
0,Housing and Living Standards,Homepage,/housingandlivingstandards,/,Housing and Living Standards,T1,
1,Work,Homepage,/work,/,Work,T1,
2,Private life and community,Homepage,/privatelifeandcommunity,/,Private life and community,T1,
3,Health,Homepage,/health,/,Health,T1,
4,Education,Homepage,/education,/,Education,T1,
5,Security and justice,Homepage,/securityandjustice,/,Security and justice,T1,


In [58]:
t2_output = pd.DataFrame(columns=output_columns)

unique_sub_topics = data[['Topic', 'Subtopic']].copy().drop_duplicates()
unique_sub_topics

for index, sub_topic  in unique_sub_topics.iterrows():
    parent_uri = '/' + to_uri(sub_topic.Topic)
    sub_topic_uri = parent_uri + '/' + to_uri(sub_topic.Subtopic)
    t2_output.loc[len(t2_output)]=[sub_topic.Subtopic, sub_topic.Topic, sub_topic_uri, parent_uri, sub_topic.Subtopic, 'T2', '']
    
t2_output.head(3)

,name,parent_name,uri,parent_uri,description,level,source
0,Housing,Housing and Living Standards,/housingandlivingstandards/housing,/housingandlivingstandards,Housing,T2,
1,Living standards,Housing and Living Standards,/housingandlivingstandards/livingstandards,/housingandlivingstandards,Living standards,T2,
2,Employment,Work,/work/employment,/work,Employment,T2,


In [57]:
t3_output = pd.DataFrame(columns=output_columns)

for index, t  in t2_output.iterrows():
    t3_output.loc[len(t3_output)]=[t[0] + '_t3', t[0], t.uri + '/1', t.uri, t.description, 'T3', '']

t3_output.head(3)

,name,parent_name,uri,parent_uri,description,level,source
0,Housing_t3,Housing,/housingandlivingstandards/housing/1,/housingandlivingstandards/housing,Housing,T3,
1,Living standards_t3,Living standards,/housingandlivingstandards/livingstandards/1,/housingandlivingstandards/livingstandards,Living standards,T3,
2,Employment_t3,Employment,/work/employment/1,/work/employment,Employment,T3,


In [61]:
t4_output = pd.DataFrame(columns=output_columns)

for index, d  in data.iterrows():
    parent_uri = '/' + to_uri(d.Topic) + '/' + to_uri(d.Subtopic) + '/1'
    uri = parent_uri + '/' + to_uri(d.Measuretitle)
    t4_output.loc[len(t4_output)] = [d.Measuretitle, d.Subtopic + '_t3', uri, parent_uri, d.Measuretitle, 'T4', '']

t4_output.head(3)

,name,parent_name,uri,parent_uri,description,level,source
0,Home ownership in England,Housing_t3,/housingandlivingstandards/housing/1/homeowner...,/housingandlivingstandards/housing/1,Home ownership in England,T4,
1,People who get financial help from their paren...,Housing_t3,/housingandlivingstandards/housing/1/peoplewho...,/housingandlivingstandards/housing/1,People who get financial help from their paren...,T4,
2,Average age of people who buy a home,Housing_t3,/housingandlivingstandards/housing/1/averageag...,/housingandlivingstandards/housing/1,Average age of people who buy a home,T4,


In [64]:
output = pd.concat([t0_output, t1_output, t2_output, t3_output, t4_output])
output.to_csv("../data/taxonomy_x.csv")